In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import tensorflow as tf
import edward as ed
import numpy as np
import six

In [3]:
import train_utils as util

In [4]:
from edward.models import Normal,Bernoulli,Categorical,Gamma,TransformedDistribution

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

In [6]:
import multiclass_BPM as mbpm

In [7]:
%matplotlib inline

In [8]:
ed.set_seed(0)

In [9]:
dataset = 'MNIST'

N = 55000 # number of training data points
N_test = 10000 # number of testing data points
#nprint = 1000
#niter = 20000
B = 2000 # batch size during training

In [10]:
if dataset == 'MNIST' or dataset == 'FASHION':
    print('MNIST')
    if dataset == 'MNIST':
        DATA_DIR = "../data/mnist/"
    else:
        DATA_DIR = "../data/fashion/" #should be downloaded before, otherwise it will automatically download MNIST
    
    mnist = input_data.read_data_sets(DATA_DIR)
    X = mnist.train.images[:N]
    Y = mnist.train.labels[:N]
    X_test = mnist.test.images[:N_test]
    Y_test = mnist.test.labels[:N_test]
    M = X.shape[1]+1
    H = max(Y)+1
    #normalise
    X = (X - X.mean())/X.std()
elif dataset == 'synthesis': 
    M = 2 # number of features
    H = 2 # number of classes
    
    s_mean = 0.
    s_std = 1.
    d_mean = 0.
    d_std = 3.
    
    Y,X = util.build_toy_dataset(mtype,N,M-1,H,s_std,s_mean,d_std)
    Y_test,X_test = util.build_toy_dataset(mtype,N_test,M-1,H,s_std,s_mean,d_std)

MNIST
Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [11]:
X = np.hstack((X,np.ones((N,1))))
X_test = np.hstack((X_test,np.ones((X_test.shape[0],1))))

In [12]:
X = X.astype(np.float32)

x_ph = tf.placeholder(tf.float32, [B,M])
y_ph = tf.placeholder(tf.int32,[B])

In [13]:
reload(mbpm)

<module 'multiclass_BPM' from 'multiclass_BPM.pyc'>

In [14]:
model = mbpm.MultiClass_BPM(N,B,H,M,x_ph,y_ph)

In [15]:
w_precision_prior = Gamma(tf.ones([H,M])*4.,tf.ones([H,M])*.5)
model.define_model(weights_mean_prior=0., weights_precision_prior=w_precision_prior)
#model.define_model()

In [16]:
model.init_inference(niter=10000,nprint=1000)

In [17]:
model.fit(X=X,Y=Y)

    1/10000 [  0%]                                ETA: 6057s | Loss: 197890.719

train_utils.py:14: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels!=None:
train_utils.py:23: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels == None:


10000/10000 [100%] ██████████████████████████████ Elapsed: 66s | Loss: 90664.102


In [18]:
#reload(mbpm)

In [19]:
py = model.predict(X_test=X_test,Y_test=Y_test)

Test accuracy:  0.8791
